# Kopp21 et al 2021 Extract-Transform-Load
**Authorship:**
Adam Klie, *08/10/2022*
***
**Description:**
Notebook to extract, transform, and load data from the Kopp21 et al (2021) dataset on JunD binding.
***

In [ ]:
import os
import sys

bin_dir = os.path.dirname(sys.executable)
os.environ["PATH"] += os.pathsep + bin_dir
from pybedtools import paths
paths._set_bedtools_path(bin_dir)
from pybedtools import BedTool

In [ ]:
output = '/cellar/users/aklie/data/eugene/kopp21/junD'

# Downloads and command line data prep
Downloaded JunD peaks (ENCFF446WOD, conservative IDR thresholded peaks, narrowPeak format), and raw DNase-seq data (ENCFF546PJU, Stam. Lab, ENCODE; ENCFF059BEU Stam. Lab, ROADMAP, bam-format) for human embryonic stem cells (H1-hesc) from the encodeproject.org and the hg38 reference genome. Alignment indices were built with samtools. Blacklisted regions for hg38 were obtained from http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/hg38-human/hg38.blacklist.bed.gz and removed using bedtools. The human genome was obtained from UCSC.

## Use `wget` to download data

In [ ]:
# Peaks from ENCODE
!wget https://www.encodeproject.org/files/ENCFF446WOD/@@download/ENCFF446WOD.bed.gz -O {output}/jund_peaks.narrowPeak.gz

# blacklisted regions to remove
!wget http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/hg38-human/hg38.blacklist.bed.gz -O  {output}/hg38.blacklisted.bed.gz
!gunzip -f  {output}/hg38.blacklisted.bed.gz

# human genome sequence hg38
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz -O  {output}/hg38.fa.gz
!gunzip -f  {output}/hg38.fa.gz
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes -O {output}/hg38.chrom.sizes

## Create the peaks to use for prediction using `bedtools`

Merge the narrow peaks

In [ ]:
BedTool(os.path.join(output, 'jund_peaks.narrowPeak.gz')).sort().merge().saveas(
    os.path.join(output, 'jund_raw_peaks.bed'))

To create the region of interest (ROI) for defining positive and negative peaks, extend the "raw" peaks by 10000bp in both directions (https://bedtools.readthedocs.io/en/latest/content/tools/slop.html).
We also need to subtract away any parts of the ROI in blacklisted regions.

In [ ]:
BedTool(os.path.join(output, 'jund_raw_peaks.bed')).slop(b=10000, 
                                                               g=os.path.join(output, 'hg38.chrom.sizes')) \
 .sort().merge().subtract(os.path.join(output, 'hg38.blacklisted.bed'))\
.saveas(os.path.join(output, 'roi_jund_extended.bed'))

This [next command](https://github.com/BIMSBbioinfo/janggu/blob/5128419cf404d8f1904d46c627c0c7963356fff1/src/janggu/janggutrim.py) trims the starts and ends of the ROIs to make them divisible by the specified window size, in this case 200

In [ ]:
# You will need to have the janggu tool installed for this
!janggu-trim {output}/roi_jund_extended.bed {output}/trim_roi_jund_extended.bed -divby 200

# SeqData preparations
Next we need to use EUGENe to read in and prep this data.

In [ ]:
import os
import eugene as eu
eu.settings.dataset_dir = '/cellar/users/aklie/data/eugene/kopp21'

In [ ]:
# Definethe input files
bed_file = os.path.join(
    eu.settings.dataset_dir,
    "jund_raw_peaks.bed" 
)
roi_file = os.path.join(
    eu.settings.dataset_dir,
    "trim_roi_jund_extended.bed"
)
refgenome = os.path.join(
    eu.settings.dataset_dir,
    "hg38.fa"
)
bed_file, roi_file, refgenome

In [ ]:
# Read in the sequences to a SeqData object. Last loading took 6m 43.1s
sdata = eu.dl.read_bed(
    bed_file=bed_file,
    roi_file=roi_file,
    ref_file=refgenome,
    dnaflank=150,
    binsize=200,
    resolution=200
)

In [ ]:
# Write this as a "raw" version of SeqData h5
sdata.write_h5sd(os.path.join(eu.settings.dataset_dir, "jund_raw.h5sd"))
sdata.ohe_seqs = sdata.ohe_seqs.transpose(0, 2, 1)

In [ ]:
# Decode the one-hot encoded sequences to save to other formats as well
sdata.seqs = eu.pp.decode_seqs(sdata.ohe_seqs)

In [ ]:
# Get the reverse complement of the sequences
eu.pp.reverse_complement_seqs_sdata(sdata)

In [ ]:
# Add in info on ranges to seq_annot
eu.pp.add_ranges_sdata(sdata)

In [ ]:
# Save combined
sdata.write_h5sd(os.path.join(eu.settings.dataset_dir, "jund_processed.h5sd"))

In [ ]:
# Split into train and test sets
eu.pp.train_test_split_sdata(
    sdata, 
    train_key="train_test",
    chr = ["chr3"]
)

In [ ]:
# Split into training and test sets
sdata_train = sdata[sdata["train_test"].values]
sdata_test = sdata[~sdata["train_test"].values]

In [ ]:
# Split the training sequences into train and validation sets
eu.pp.train_test_split_sdata(
    sdata_train,
    train_key="train_val",
    chr=["chr2"]
)

In [ ]:
# Save train
sdata_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jund_train_processed.h5sd"))

In [ ]:
# Save test
sdata_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jund_test_processed.h5sd"))

---

# Scratch